In [1]:
from sodapy import Socrata
from plotly import graph_objects as go
from datetime import datetime, date, time, timedelta
from datetime import datetime
# from highcharts import Highchart
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import plotly.express as px
import datetime
import numpy as np
import utils
import plotly
from ipywidgets import widgets
import ipywidgets
from ipywidgets import interact
%matplotlib inline

from matplotlib.dates import DateFormatter
pd.options.display.max_colwidth = 200
import warnings
warnings.filterwarnings('ignore')

print('import done')

import done


### Get Data from Socrata

In [2]:
## Build client
usrname = "jjensen@mointern.nyc.gov"
pword = "Timspw19!"
client = Socrata("data.cityofnewyork.us","TnnbmTZvKbFz7tJ4NpzkfUG6C", username=usrname, password=pword)

## Make API calls
# auth_json_downloads = client.get("beb8-u7wp", limit=2000000)
# auth_json_dailyviews = client.get("qiif-mxcx", limit=50000)
auth_json_authentications = client.get("8aih-pmax", limit = 20000)
auth_json_datasetfacts = client.get("gzid-z3nh", limit = 50000)
auth_json_public_inventory = client.get("5tqd-u88y", limit=5000)
auth_json_api_calls = client.get("u7pi-wq4y", limit=2000)
auth_json_the_plan = client.get("qj2z-ibhs", limit=5000)


## Get the private asset inventory
## only necessary because the columns `asset_type` is only in the private and not the public asset inventory
auth_json_asset_inventory = client.get("r8cp-r4rc", limit = 40000)

### Generate base dataframes to build off of throughout the notebook

In [3]:
## Convert json files into pandas dataframes
# dv = pd.DataFrame(auth_json_dailyviews)
# dauth = pd.DataFrame(auth_json_authentications)
dfacts = pd.DataFrame(auth_json_datasetfacts)
# ddownloads = pd.DataFrame(auth_json_downloads)
public = pd.DataFrame(auth_json_public_inventory)
private = pd.DataFrame(auth_json_asset_inventory)
the_plan = pd.DataFrame(auth_json_the_plan)

### Data Cleaning

In [4]:
# Create merged_filter, the dataframe that has only assets defined as datasets
filter_list = ['dataset','filter', 'gis map']
## Cleaning dates entered in the wrong format
bad_index_list = list(public.query('date_made_public == "August 9, 2-019"').index)
for val in bad_index_list:
    public.at[val, 'date_made_public'] = '8/9/2019'

# need to merge in private so that we can get the asset type
dupes = private[['u_id', 'agency', 'type','name', 'parent_uid','derived_view']]

public = utils.date_transformation(public, 'date_made_public')
upUntilfy20 = utils.apply_date_mask(public, 'date_made_public', fy=False)
merged_total = pd.merge(public, dupes, how='left', on='u_id')
merged_filter = merged_total[merged_total.type.isin(filter_list)]


# There are only 87 derived assets, so our total number of datasets would greatly increase 
# merged_total[['name_x', 'derived_view']].query('derived_view == "true"')

## Derived vs Parent Assets

In [31]:


not_derived = merged_total.query('derived_view == "false"')
not_derived.shape
not_derived[~not_derived.type.isin(filter_list)][['name_x','agency_x','type','u_id']]
num_datasets = merged_filter.groupby('derived_view').size()['false']
print('A dataset is defined as an asset where derived_view is False and asset_type is either gis map, dataset, or filter.')

print('There are {} datasets on the NYC Open Data Portal.'.format(num_datasets))

A dataset is defined as an asset where derived_view is False and asset_type is either gis map, dataset, or filter.
There are 2208 datasets on the NYC Open Data Portal.


In [36]:
dataset_df = not_derived[not_derived.type.isin(filter_list)][['name_x','agency_x','type','u_id','date_made_public']]

## Number of Datasets on NYC Open Data Portal

In [48]:
dataset_df.sort_values(by='date_made_public', ascending=False)
null_public = dataset_df[dataset_df.date_made_public.isnull()].shape[0]
print('There are {} datasets without a value for date_made_public'.format(null_public))

There are 14 datasets without a value for date_made_public


In [37]:
published_datasets_by_fy = utils.create_fy_columns(dataset_df, 'date_made_public')
published_assets_by_fy = utils.create_fy_columns(dataset_df, 'date_made_public')

In [38]:
num_datasets = pd.DataFrame(published_datasets_by_fy[['until_fy_16', 'until_fy_17', 'until_fy_18', 'until_fy_19', 'until_fy_20']].sum())\
    .reset_index().rename(columns={'index':'FY', 0:'Number of Datasets on ODP'})

num_assets = pd.DataFrame(published_assets_by_fy[['until_fy_16', 'until_fy_17', 'until_fy_18', 'until_fy_19', 'until_fy_20']].sum())\
    .reset_index().rename(columns={'index':'FY', 0:'Number of Assets on ODP'})

assets_by_type = pd.DataFrame(merged_total.groupby('type').size())\
    .reset_index().sort_values(ascending=False, by=0)\
    .rename(columns={'type':'asset_type', 0:'Count'})

In [57]:


fig = go.Figure(data=[go.Bar(
    x=num_datasets['FY'],
    y=num_datasets['Number of Datasets on ODP'],
    text=num_datasets['Number of Datasets on ODP'],
    textposition='auto')])

fig.update_layout(
     title=go.layout.Title(
        text="Total Datasets at the End of the Fiscal Year",
        xref="paper",
        x=.5)
    )
fig.show()

In [56]:
# Number of Assets per FY

fig = go.Figure(data=[go.Bar(
    x=num_assets['FY'],
    y=num_assets['Number of Assets on ODP'],
    text=num_assets['Number of Assets on ODP'],
    textposition='auto')])
fig.update_layout(
     title=go.layout.Title(
        text="Number of Assets at the End of the Fiscal Year",
        xref="paper",
        x=.5)
    )
fig.show()

In [54]:

fig = go.Figure(data=[go.Bar(
    x=assets_by_type['asset_type'],
    y=assets_by_type['Count'],
    text=assets_by_type['Count'],
    textposition='auto')])

fig.update_layout(
     title=go.layout.Title(
        text="Total Assets by Asset Type",
        xref="paper",
        x=.5)
    )
fig.show()

In [11]:
# Number of Unique Agencies that Published Dataset per Fiscal Year

published_datasets_by_fy.groupby(['fy_16','fy_17','fy_18','fy_19'], as_index=True)\
    ['agency_x'].nunique().reset_index()

,fy_16,fy_17,fy_18,fy_19,agency_x
0,0,0,0,0,63
1,0,0,0,1,39
2,0,0,1,0,35
3,0,1,0,0,32
4,1,0,0,0,33


# Automations

In [12]:
cumulative_automations = pd.DataFrame(merged_filter.query('automation == "Yes" & date_made_public')\
    .groupby(pd.Grouper(key='date_made_public', freq=('M'))).size().cumsum()).rename(columns={0:'Count'}).last('18M')

fig = go.Figure(data=[go.Bar(
    x=cumulative_automations.index,
    y=cumulative_automations['Count'],
    text=cumulative_automations['Count'],
    textposition='auto')])

fig.update_layout(
     title=go.layout.Title(
        text="Cumulative Total Automated Datasets in the Last 18 Months",
        xref="paper",
        x=.5)
     )
fig.show()

In [13]:
public['last_update_date_data'] = pd.to_datetime(public['last_update_date_data'])
merged_filter['last_update_date_data'] = pd.to_datetime(public['last_update_date_data'])
merged_filter['last_update_date_data'] = merged_filter['last_update_date_data'].dt.tz_convert(None)

In [14]:
# merged filter is a copy of the asset inventory that is filtered to only include assets defined as datasets
freshness_df = merged_filter[['name_x','agency_x','u_id','update_frequency',\
                              'date_made_public','last_update_date_data','automation']]

# Remove datasets with update frequencies for which we cannot determine freshness
freshness_df = freshness_df[~freshness_df['update_frequency'].isin(['Historical Data', 'As needed'])].reset_index()
freshness_df.rename(columns={'agency_x':'agency'}, inplace=True)

# Removes automated datasets 
# freshness_df = freshness_df[freshness_df['automation'] != "Yes"]

In [15]:
FREQ_DICTIONARY = {
    
    'Daily' : [np.timedelta64(1, 'D')],
    'Weekly' : [np.timedelta64(1, 'W')],
    'Biweekly ': [np.timedelta64(4, 'D')],
    'Monthly': [np.timedelta64(1,'M')],
    'Quarterly':[np.timedelta64(3, 'M')],
    '2 to 4 times per year' : [np.timedelta64(6, 'M')],
    'Triannually': [np.timedelta64(4, 'M')],
    'Biannually ': [np.timedelta64(6, 'M')],
    'Annually': [np.timedelta64(1, 'Y')],
    'Weekdays': [np.timedelta64(2, 'D')],
    'Hourly': [np.timedelta64(1, 'h')],
    'Several times per day': [np.timedelta64(1, 'D')]
}

def calculate_freshness(df, date_col, fresh_col):
    '''
    Returns the asset inventory dataframe with an additional true/false column that
    indicates whether or not a dataset is fresh. Fresh is defined as having a 
    last_update_date_data timestamp within the stated update frequency
    
    Inputs:
        df: a dataframe, one that has filtered inapplicable update frequencies
            and all automated datasets
        date_col: last updated timestamp
        fresh_col: name of the binary fresh or stale column
    Returns:
        dataframe indicating if each dataset is stale or fresh
    '''
    
    df[fresh_col] = np.nan
    
    for frequency in df.update_frequency.unique():
                    
        
        temp = df.query('update_frequency == "{}"'.format(frequency))  
        today = datetime.datetime.today()
        idx = temp.index
        
#         if frequency == 'Weekdays':
#             np.busday_count(today, temp['last_update_date_data'])
            
        df.loc[idx, fresh_col] = (today - temp['last_update_date_data']) < FREQ_DICTIONARY[frequency][0]
    
    return df

## Measuring Data Freshness 

In [16]:
# Percent of datasets fresh or stale
fresh_df = calculate_freshness(freshness_df, 'last_update_date_data', 'fresh')
fresh_df['agency'] = fresh_df['agency'].astype(str)

# get index value for all automated datasets
auto_idx = np.where(freshness_df['automation']=='Yes')

# update fresh_df to be true for all automated datasets
fresh_df.fresh.iloc[auto_idx] = True

pct_fresh = fresh_df.groupby('fresh').size().reset_index().rename(columns={0:'count'})
pct_fresh['pct'] = pct_fresh['count'].apply(lambda x: x/pct_fresh['count'].sum())
# print(pct_fresh)
print('\n\n')

# Stale datasets by update_frequency
uf = fresh_df.query('fresh == False').groupby('update_frequency')\
    .size().reset_index().rename(columns={0:'Count'}).sort_values(by='Count')

# 324 are annual, which could mean they're mistagged and should be historical
    
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']


labels = uf['update_frequency']
values = uf['Count']

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.update_layout(
     title=go.layout.Title(
        text="Stale Datasets According to their Update Frequency",
        xref="paper",
        x=.5)
     )
           
fig.show()

   fresh  count       pct
0  False    566  0.444619
1   True    707  0.555381





In [17]:
stacked = fresh_df.groupby('agency')['fresh'].value_counts()\
    .unstack().fillna(0).reset_index().rename(columns={False:'Stale',True:'Fresh'})\
    .sort_values(by='Fresh',ascending=False)

fig = go.Figure(data=[
    go.Bar(name='Stale', x=stacked['agency'], y=stacked['Stale']),
    go.Bar(name='Fresh', x=stacked['agency'], y=stacked['Fresh'])
])

fig.update_layout(
     title=go.layout.Title(
        text="How Stale Is Each Agency's Data?",
        xref="paper",
        x=.5)
     )

fig.update_xaxes(showticklabels=False)
# Change the bar mode
fig.update_layout(barmode='stack')

fig.update_layout(
    autosize=False,
    width=1000,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),

)
fig.show()

In [18]:
ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [19]:
output = widgets.Output()
plot_output = widgets.Output()
output_agency = widgets.Output()

dropdown_agency = widgets.Dropdown(options = unique_sorted_values_plus_ALL(fresh_df.agency))


def common_filtering(agency):
    output.clear_output()
    output_agency.clear_output()
    plot_output.clear_output()
    
    if (agency == ALL):
        common_filter = fresh_df
    else:
        common_filter = fresh_df[(fresh_df.agency == agency)]
        
    with plot_output:

        df = common_filter.fresh.value_counts().reset_index().rename(columns={'index':'status','fresh':'count'})

        colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

        labels = df['status']
        values= df['count']

        fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

        fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                          marker=dict(colors=colors, line=dict(color='#000000', width=2)))
        fig.update_layout(
            title=go.layout.Title(
            text="The Datasets Are Up-to-Date",
            xref="paper",
            x=.5)
        )
        fig.show()
        
    with output_agency:
        with pd.option_context("display.max_rows", 1000):
            if (agency == ALL):
                display(fresh_df)
            else:
                display(fresh_df[fresh_df.agency == agency])

def dropdown_agency_eventhandler(change):
    common_filtering(change.new)

dropdown_agency.observe(dropdown_agency_eventhandler, names='value')

display(dropdown_agency)

display(plot_output)

display(output_agency)

Dropdown(options=('ALL', '311', 'Board of Standards and Appeals (BSA)', 'Business Integrity Commission (BIC)',…

Output()

Output()

In [20]:
## data cleanup
make_int = ['asset_columns', 'asset_rows']

for col in make_int:
    dfacts[col] = pd.to_numeric(dfacts[col])

dfacts['date'] = pd.to_datetime(dfacts['date'])

tables = dfacts[dfacts.derived_asset_type.isin(["table"])]
na = dfacts.query('derived_asset_type == "n/a"')
grouped = dfacts.groupby('asset_title')

dnum_rows = {}

for name, group in grouped:
    temp = group[group['date'] == group['date'].max()].reset_index()
    num_rows = temp.asset_rows[0]
    agency = temp['agency'][0]
    
    dnum_rows[agency] = dnum_rows.get(agency, 0) + num_rows

In [21]:
clean_count = merged_filter.groupby('agency_x').size().sort_values(ascending=True)\
    .reset_index().rename(columns={'agency_x':'agency', 0:'count'})

df = pd.DataFrame.from_dict(dnum_rows, orient='index').rename_axis('agency').reset_index().rename(columns={0:'count'})

clean_count.columns = ['agency', 'numdatasets']
df.columns = ['agency', 'numrows']
rv = pd.merge(df, clean_count, on='agency')

rv['lognumrows'] = rv['numrows'].apply(lambda x: math.log(x))

## Publishing Data

In [49]:
fig = go.Figure(data=go.Scatter(x=rv['numdatasets'],
                                y=rv['lognumrows'],
                                mode='markers',
                                marker=dict(
                                    color=rv['numdatasets'], #set color equal to a variable
                                    colorscale='Jet', # one of plotly colorscales
                                    showscale=False
                                ),
#                                 marker_color=rv['lognumrows'],
                                text=rv['agency'])) # hover text goes here

fig.update_layout(
     title=go.layout.Title(
        text="So Who's *Really* Publishing?",
        xref="paper",
        x=.5),
    
     xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Total Number of Datasets",
            font=dict(
#                 family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Log Number of Total Rows",
            font=dict(
#                 family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)
fig.update_layout(
    autosize=False,
    width=1000,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
#     paper_bgcolor="LightSteelBlue",
)

fig.show()